Ground Truth Image Set Capture Conditions 
- 24 photos of the same Post-it note was taken using UTC's imaging aparatus.
- A metal washer was used to keep the non-adhesive side of the post-it flush.
- Images were taken at varying focal distances.
- Distance measurements are according to kiser Copy Stand's arm measurements and are not accurate to actual  focal distances due to the photo box's stage height offset.
    - ranged from 82cm - 61cm
    - photo box height offset is constant and makes focal distance closer by (need to take measurement) XXcm.
    - it was not possible to retain the color reference in images closer than 61cm.

- Post-it notes are 7.62 cm x 7.62 cm

In [34]:
import numpy as np
from math import sqrt
import cv2
from glob import glob
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
plt.rcParams['figure.figsize'] = [25, 25]

import piexif
import json

#### Identify the image test set

In [35]:
imgs = glob("./scale_test_set/processed/*.jpg")
len(imgs)

24

#### Extract the scale values for each image as determined by HerbASAP

In [64]:
def cvt_num(x):
    try:
        x = float(x)
    except:
        x = np.nan
    return x

def cvt_ci(scale, ci):
    scale = cvt_num(scale)
    ci = cvt_num(ci)
    try:
        if (ci == scale) or (ci > 50):
            ci = np.nan
            scale = np.nan
        else:
            ci = float(ci)
            scale = float(scale)
    except:
        ci = np.nan
        scale = np.nan
    return scale, ci

def extract_id(path):
    img_id = path.split("/")[-1].split(".jpg")[0]
    return img_id

def remove_vignette(img, correction_factor=0.75):
    corrections = cv2.imread("./scale_test_set/vignette_correction1.jpg")
    corrections = (corrections * correction_factor) // 255
    corrections = corrections.astype('uint8')
    img = (img + corrections)
    img = cv2.normalize(img,  img, 0, 255, cv2.NORM_MINMAX)

    # verify results are as expected
    #cv2.imwrite("corrected.jpg", img)
    return img

def scale_contour(cnt, scale):
    # taken from:
    # https://medium.com/analytics-vidhya/tutorial-how-to-scale-and-rotate-contours-in-opencv-using-python-f48be59c35a2
    M = cv2.moments(cnt)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])

    cnt_norm = cnt - [cx, cy]
    cnt_scaled = cnt_norm * scale
    cnt_scaled = cnt_scaled + [cx, cy]
    cnt_scaled = cnt_scaled.astype(np.int32)

    return cnt_scaled

def postit_pxs(imgpath):
    """adapted from: https://stackoverflow.com/questions/44588279/find-and-draw-the-largest-contour-in-opencv-on-a-specific-color-python"""
    im = cv2.imread(imgpath)
    im = remove_vignette(im)

    # crop 20% of image's height to remove crc and 10% of image's width to remove guides
    l, w, _ = im.shape
    im = im[l//10:l - l//10, w//20: w - w//20, ...]
    image = im
    # establish color boundries for the post-it
    lower = [150, 150, 150]
    upper = [255, 255, 255]

    # create NumPy arrays from the boundaries
    lower = np.array(lower, dtype="uint8")
    upper = np.array(upper, dtype="uint8")

    # find the colors within the specified boundaries and apply
    mask = cv2.inRange(image, lower, upper)
    output = cv2.bitwise_and(image, image, mask=mask)

    cont_image = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
    contours, hierarchy = cv2.findContours(cont_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    if len(contours) != 0:
        contour = max(contours, key = cv2.contourArea)
        # generate dummy image to paint the contour onto
        contour_mask = np.zeros(cont_image.shape, np.uint8)
        cv2.drawContours(contour_mask, [contour], -1, (255,255,255), -1)
        
        area = np.sqrt(cv2.countNonZero(contour_mask))
        #area = cv2.contourArea(contour)

        # draw the identified contour in red
        #cv2.drawContours(output, contour, -1, (50, 50, 255) , 5)
        #cv2.imwrite(f'{extract_id(imgpath)}_contour.jpg',contour_mask)
    else:
        area = False
    return area

results = {}
#for imgpath in ["./scale_test_set/processed/59.jpg"]:
for imgpath in set(imgs):
    img_id = extract_id(imgpath)
    exifDict = piexif.load(imgpath)
    desc = json.loads(exifDict["0th"][270])
    pixelsPerMM, CI = cvt_ci(desc["pixelsPerMM"], desc["pixelsPerMMConfidence"])

    postit_area_px = postit_pxs(imgpath)
    p_area_mm = round(postit_area_px / pixelsPerMM, 3)
    p_area_ci_min = round(postit_area_px / (pixelsPerMM + CI), 3)
    p_area_ci_max = round(postit_area_px / (pixelsPerMM - CI), 3)
    
    
    # results organized as list under key. 
    
    results[img_id] = [pixelsPerMM, CI, postit_area_px, p_area_mm, p_area_ci_min, p_area_ci_max]

#### Desired output ~ 76.2mm^2


post-it notes are 3" x 3" (7.62 cm x 7.62 cm)

In [66]:
# for #74, gimp px count is 1089051
ids = list(results.keys())
ids.sort()

areas = []
deviations = []
deviation_pcts = []
is_contained_in_ci = []
for id in ids:
    print(f"{id}: {results.get(id)}")
    pixelsPerMM, CI, postit_area_px, p_area_mm, p_area_ci_min, p_area_ci_max = results.get(id)

    deviation = p_area_mm - 76.2
    deviations.append(deviation)
    deviation_pct = abs(deviation) / 76.2 * 100
    deviation_pcts.append(deviation_pct)
    
    # check if CI contains expected value
    if (p_area_ci_min <= 76.2 <= p_area_ci_max):
        is_contained = True
    else:
        is_contained = False
    is_contained_in_ci.append(is_contained)
    print(is_contained)
    
    areas.append(p_area_mm)
# List order results are:[pixelsPerMM, CI, postit_area_px, p_area_mm, p_area_ci_min, p_area_ci_max]

avg_area = np.mean(areas)
avg_area_diff = avg_area - 76.2
pct_contained = np.round(is_contained_in_ci.count(True) / len(is_contained_in_ci) *100, 3)

print()
print(f"mean difference from 76.2 = {avg_area_diff}")
print(f"deviation pcts (either direction) = {np.round(np.mean(deviation_pcts), 3)}%")
print(f"pct of results containing 76.2  in ci = {pct_contained}%")

59: [18.608, 0.336, 1392.069322986467, 74.81, 73.483, 76.186]
False
60: [17.959, 0.306, 1362.8151011784394, 75.885, 74.613, 77.2]
True
61: [17.822, 0.309, 1332.6188502343796, 74.774, 73.499, 76.093]
False
62: [17.25, 0.247, 1304.9697314497375, 75.65, 74.582, 76.749]
True
63: [17.044, 0.367, 1279.4811448395792, 75.069, 73.487, 76.721]
True
64: [16.601, 0.2, 1254.9513934810384, 75.595, 74.695, 76.517]
True
65: [16.177, 0.219, 1228.2788771284802, 75.927, 74.913, 76.969]
True
66: [15.988, 0.256, 1204.8103585212073, 75.357, 74.17, 76.583]
True
67: [15.659, 0.207, 1182.501162790126, 75.516, 74.531, 76.527]
True
68: [15.153, 0.187, 1160.4210442766023, 76.58, 75.647, 77.537]
True
69: [14.965, 0.273, 1139.4718074616853, 76.142, 74.778, 77.557]
True
70: [14.652, 0.232, 1118.9052685549389, 76.365, 75.175, 77.594]
True
71: [14.379, 0.228, 1098.8730590928144, 76.422, 75.229, 77.653]
True
72: [14.076, 0.227, 1080.0911998530494, 76.733, 75.515, 77.991]
True
73: [13.831, 0.231, 1060.9509885004113, 76.

In [67]:
deviations

[-1.3900000000000006,
 -0.3149999999999977,
 -1.426000000000002,
 -0.5499999999999972,
 -1.1310000000000002,
 -0.605000000000004,
 -0.27299999999999613,
 -0.8430000000000035,
 -0.6839999999999975,
 0.37999999999999545,
 -0.058000000000006935,
 0.16499999999999204,
 0.2219999999999942,
 0.5330000000000013,
 0.5079999999999956,
 0.32399999999999807,
 0.5859999999999985,
 0.3049999999999926,
 0.664999999999992,
 1.7590000000000003,
 1.5789999999999935,
 1.1550000000000011,
 1.1469999999999914,
 1.0969999999999942]

In [68]:
deviation_pcts

[1.8241469816272973,
 0.4133858267716505,
 1.871391076115488,
 0.7217847769028835,
 1.4842519685039373,
 0.7939632545931811,
 0.358267716535428,
 1.10629921259843,
 0.8976377952755873,
 0.4986876640419888,
 0.07611548556431355,
 0.21653543307085568,
 0.29133858267715773,
 0.6994750656167995,
 0.6666666666666609,
 0.42519685039369826,
 0.7690288713910741,
 0.4002624671915913,
 0.8727034120734803,
 2.3083989501312336,
 2.0721784776902803,
 1.5157480314960643,
 1.5052493438320096,
 1.439632545931751]

old results for holding

In [58]:
# for #74, gimp px count is 1089051
ids = list(results.keys())
ids.sort()

areas = []
deviations = []
deviation_pcts = []
is_contained_in_ci = []
for id in ids:
    print(f"{id}: {results.get(id)}")
    pixelsPerMM, CI, postit_area_px, p_area_mm, p_area_ci_min, p_area_ci_max = results.get(id)

    deviation = p_area_mm - 5806
    deviations.append(deviation)
    deviation_pct = np.round(abs(deviation) / 5806 * 100, 3)
    deviation_pcts.append(deviation_pct)
    
    # check if CI contains expected value
    if (p_area_ci_min <= 5806 <= p_area_ci_max):
        is_contained = True
    else:
        is_contained = False
    is_contained_in_ci.append(is_contained)
    print(is_contained)
    
    areas.append(p_area_mm)
# List order results are:[pixelsPerMM, CI, postit_area_px, p_area_mm, p_area_ci_min, p_area_ci_max]

avg_area = np.mean(areas)
avg_area_diff = avg_area - 5806
pct_contained = np.round(is_contained_in_ci.count(True) / len(is_contained_in_ci) *100, 3)

print()
print(f"mean difference from 5806 = {avg_area_diff}")
print(f"deviation pcts (either direction) = {np.round(np.mean(deviation_pcts))}%")
print(f"pct of results containing 5806 in ci = {pct_contained}%")

59: [18.608, 0.336, 1937857, 5596.57504071881, 5399.808472660531, 5804.2960135765115]
False
60: [17.959, 0.306, 1857265, 5758.502684938135, 5567.170490652677, 5959.870622706931]
True
61: [17.822, 0.309, 1775873, 5591.12273252268, 5402.171763255731, 5790.163256100846]
False
62: [17.25, 0.247, 1702946, 5722.985927326192, 5562.546995499116, 5890.467540818784]
True
63: [17.044, 0.367, 1637072, 5635.399764079562, 5400.33062490679, 5886.158101315195]
True
64: [16.601, 0.2, 1574903, 5714.593082432393, 5579.349235708373, 5854.814679594344]
True
65: [16.177, 0.219, 1508669, 5764.982530505922, 5612.006266471076, 5924.300034300151]
True
66: [15.988, 0.256, 1451568, 5678.7023592688265, 5501.123866939054, 5865.020171007007]
True
67: [15.659, 0.207, 1398309, 5702.6288215579725, 5554.797777027378, 5856.440751713037]
True
68: [15.153, 0.187, 1346577, 5864.53990585541, 5722.429792159976, 6012.010224645862]
True
69: [14.965, 0.273, 1298396, 5797.673131727679, 5591.794858154798, 6015.134317288305]
True
7